This notebook is a supplement to the paper "Negative events - Logical and philosophical aspects"[link] by Lucas Champollion[link] and Timothée Bernard[link].

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s interface to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations.

## Outline

In [33]:
from nltk.test.inference_fixt import setup_module

setup_module()

from nltk import *
#from nltk.sem.drt import DrtParser

from nltk.sem.logic import *
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## Axioms

### Lattice axioms

In [34]:
part_def = read_expr('Partof(x,y) <-> Product(x, y) = x')
proper_part_def = read_expr('Properpartof(x,y) <-> Partof(x,y) & -(x=y)')

sum_id = read_expr('Sum(x,x) = x')
product_id = read_expr('Product(x,x) = x')

sum_comm = read_expr('Sum(x,y) = Sum(y,x)')
product_comm = read_expr('Product(x,y) = Product(y,x)')

sum_assoc = read_expr('Sum(x,Sum(y,z)) = Sum(Sum(x,y),z)')
product_assoc = read_expr('Product(x,Product(y,z)) = Product(Product(x,y),z)')

sum_absorp = read_expr('Sum(x,Product(x,y)) = x')
product_absorp = read_expr('Product(x,Sum(x,y)) = x')

lattice = [part_def, proper_part_def, sum_id, product_id, sum_comm, product_comm, sum_assoc, product_assoc, sum_absorp, product_absorp]

### Bounded lattice axioms

In [35]:
bot = read_expr('Sum(x, Bot) = x')
top = read_expr('Product(x, Top) = x')

bounded_lattice = lattice + [bot, top]

### Distributive lattice axioms

In [36]:
dist = read_expr('Product(x,Sum(y,z)) = Sum(Product(x,y), Product(x,z))')

dist_bounded_lattice = bounded_lattice + [dist]

### Complete lattice axioms

In [37]:
uniq_sum = read_expr('all P.(exists z.((all x.(P(x) -> Partof(x,z))) & all y.((all x.(P(x) -> Partof(x,y))) -> Partof(z,y))))')
uniq_product = read_expr('all P.(exists z.((all x.(P(x) -> Partof(z,x))) & all y.((all x.(P(x) -> Partof(y,x))) -> Partof(y,z))))')
# this only guarantees existence and not uniqueness, 
# but sums and products are unique wherever they exist

### Boolean algebra axioms

In [38]:
complemented = read_expr('all x.(exists y. (Product(x, y) = Bot & Sum(x, y) = Top))')

#sectional complement
sec_compl = read_expr('all x. (all y. (exists z. (Sum(x, z) = y & Product(x, z) = Bot)))')

boolean_algebra = dist_bounded_lattice + [uniq_sum, uniq_product, complemented]

### Exclusion axioms

In [39]:
excl_cum = read_expr('Excl(x1, y1) & Excl(x2, y2) -> Excl(Sum(x1, x2), Sum(y1, y2))')
excl_symm = read_expr('Excl(x, y) <-> Excl(y, x)')

### Possibility axioms

In [42]:
#define conflict:
confl_def = read_expr('Confl(x, y) <-> exists z1.(Partof(z1,x) & exists z2.(Partof(z2,y) & Excl(z1, z2)))')

#define possibilities:
#int_poss_def = read_expr('int_poss(x) <-> -Excl(x, x)')
#ext_poss_def = read_expr('ext_poss(x) <-> exists y.(int_poss(y) & -exists z.(Product(y, z) = y & int_poss(z) & -(confl(z,x))))')

poss_def = read_expr('Poss(x) <-> -Confl(x, x)')

#define world:
#world_def = read_expr('world(x) <-> (int_poss(x) & all y.((Product(x, y) = y) -> -int_poss(y)))')

world_def = read_expr('World(x) <-> (Poss(x) & all y.(Properpartof(x,y) -> -Poss(y)))')

#actual world
#actual_world = read_expr('world(actual_world)')

#define actual:
#actual_def = read_expr('actual(x) <-> Product(x, actual_world) = x')

#Harmony
#poss_harm = read_expr('all x.(-int_poss(x) -> -ext_poss(x))')

poss_harm = read_expr('all x.(all y. ((World(x) & -Confl(x, y)) -> Poss(y)))')

#Nirvana
poss_nirv = read_expr('all x.(-Confl(Bot, x))')

#Rashōmon
#poss_rshm = read_expr('all x y.((int_poss(x) & int_poss(y) & -confl(x, y)) -> int_poss(Product(x,y)))')
poss_rshm = read_expr('all x y.((Poss(x) & Poss(y) & -Confl(x, y)) -> Poss(Sum(x,y)))')


#Cosmopolitanism
#poss_cosm = read_expr('all x.(int_poss(x) -> exists y.(world(y) & Product(x, y) = y))')

poss_cosm = read_expr('all x.(Poss(x) -> exists y.(World(y) & Partof(x,y)))')

poss_ax = [poss_harm, poss_nirv, poss_rshm, poss_cosm]

#Construct event frame
#choice point: definition of possibility
#choice point: formulation of harmony

event_space = boolean_algebra + [excl_cum, excl_symm, confl_def, poss_def, world_def]

event_frame = event_space + poss_ax

In [55]:
def try_to_derive(axiom):
    goal = axiom
    prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == axiom)], timeout = 1000)
    prover.prove()
    print(prover.proof())

def try_to_show_independence(axiom):
    goal = axiom
    mb = MaceCommand(goal, assumptions = [ax for ax in event_frame if not (ax == axiom)])
    print(mb.build_model())
    print(mb.model(format='cooked'))

## Independence of axioms

In [53]:
try_to_derive(poss_harm) # times out

KeyboardInterrupt: 

In [56]:
try_to_show_independence(poss_harm) # succeeds

True
% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 0.

c2 = 1.

f1(0) = 0.
f1(1) = 0.

f3(0) = 0.
f3(1) = 0.

f5(0) = 1.
f5(1) = 0.

f8(0) = 0.
f8(1) = 0.

f9(0) = 0.
f9(1) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(1,0) = 0.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(1,0) = 1.
Sum(1,1) = 1.

f2(0,0) = 0.
f2(0,1) = 0.
f2(1,0) = 0.
f2(1,1) = 0.

f4(0,0) = 0.
f4(0,1) = 0.
f4(1,0) = 0.
f4(1,1) = 0.

f6(0,0) = 0.
f6(0,1) = 0.
f6(1,0) = 0.
f6(1,1) = 1.

f7(0,0) = 0.
f7(0,1) = 0.
f7(1,0) = 0.
f7(1,1) = 1.

  P(0).
- P(1).

  Poss(0).
- Poss(1).

  World(0).
- World(1).

- Confl(0,0).
- Confl(0,1).
- Confl(1,0).
  Confl(1,1).

- Excl(0,0).
- Excl(0,1).
- Excl(1,0).
  Excl(1,1).

  Partof(0,0).
  Partof(0,1).
- Partof(1,0).
  Partof(1,1).

- Properpartof(0,0).
  Properpartof(0,1).
- Properpartof(1,0).
- Properpartof(1,1).



In [60]:
try_to_show_independence(poss_cosm)

KeyboardInterrupt: 

In [ ]:
try_to_derive(poss_cosm)

In [43]:
#Harmony is derivable from other possibility axioms

goal = poss_harm
prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == poss_harm)], timeout = 1000)
prover.prove()
print(prover.proof())

KeyboardInterrupt: 

In [ ]:
#Nirvana is derivable from other possibility axioms (no counterexample found)
goal = poss_nirv
mb = MaceCommand(goal, assumptions = [ax for ax in event_frame if not (ax == poss_nirv)])
mb.build_model()

In [ ]:
goal = poss_nirv
prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == poss_nirv)], timeout = 1000)
prover.prove()
print(prover.proof())

In [ ]:
#Rashōmon is derivable from other possibility axioms
goal = poss_rshm
prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == poss_rshm)], timeout = 1000)
prover.prove()
print(prover.proof())

In [49]:
#Cosmopolitan is not derivable from other possibility axioms (counterexample found)
goal = poss_cosm
mb = MaceCommand(goal, assumptions = [ax for ax in event_frame if not (ax == poss_cosm)])
mb.build_model()

False

In [ ]:
#Cosmopolitan is derivable from other possibility axioms
goal = poss_cosm
prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == poss_cosm)], timeout = 1000)
prover.prove()
print(prover.proof())

## Theorems

### Plenitude

In [41]:
#check plenitude
#goal = read_expr('all x.(int_poss(x) -> ext_poss(x))')

goal = read_expr('all x.((exists y.(-Confl(x, y) & World(y))) <-> Poss(x))')

#goal = read_expr('all x.(Poss(x) <-> (exists y.(-Confl(x, y) & World(y))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 12513 was started by champollion on standingdesk.home,
Fri Jul  8 15:37:17 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 11.69 (+ 0.45) seconds.
% Length of proof is 56.
% Level of proof is 13.
% Maximum clause weight is 15.000.
% Given clauses 6932.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
7 Excl(x,y) <-> Excl(y,x).  [assumption].
8 Confl(x,y) <-> (exists z (Partof(z,x) & (exists u (Partof(u,y) & Excl(z,u))))).  [assumption].
9 Poss(x) <-> -Confl(x,x).  [assumption].
10 World(x) <-> Poss(x) & (all y (Properpartof(x,y) -> -Poss(y))).  [assumption].
11 (all x all y (World(x) & -Confl(x,y) -> P

### Distributivity of actuality

In [14]:
#check distributivity of actuality
goal = read_expr('all x.((exists y. Actual(y) & Partof(x,y)) -> Actual(x))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

KeyboardInterrupt: 

In [ ]:
#Find counterexample for distributivity of actuality - fail

goal = read_expr('all x.((exists y. Actual(y) & Product(x, y) = x) -> Actual(x))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

### Cumulativity of actuality

In [ ]:
#check cumulativity of actuality
goal = read_expr('all P.((all x.(P(x) -> Actual(x))) -> (exists z.(all x.(P(x) -> Product(x, z) = x) & all y.((all x.(P(x) -> Product(x, y) = x)) -> Product(y,z) = z)) & Actual(z)))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

In [ ]:
#find counterexample for cumulativity of actuality - fail

goal = read_expr('all P.((all x.(P(x) -> Actual(x))) -> (exists z.(all x.(P(x) -> Product(x, z) = x) & all y.((all x.(P(x) -> Product(x, y) = x)) -> Product(y,z) = z)) & Actual(z)))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

### Manichaeism

In [24]:
#check Manichaeism
goal = read_expr('all x.((World(x) -> all y.(Partof(y,x) | Confl(x, y))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 12229 was started by champollion on standingdesk.home,
Fri Jul  8 15:14:13 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.06 (+ 0.01) seconds.
% Length of proof is 30.
% Level of proof is 7.
% Maximum clause weight is 11.000.
% Given clauses 413.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
2 Properpartof(x,y) <-> Partof(x,y) & x != y.  [assumption].
10 World(x) <-> Poss(x) & (all y (Properpartof(x,y) -> -Poss(y))).  [assumption].
11 (all x all y (World(x) & -Confl(x,y) -> Poss(y))).  [assumption].
13 (all x all y (Poss(x) & Poss(y) & -Confl(x,y) -> Poss(Sum(x,y)))).  [assumption].
15 (all x (Wo

### Classicality of conjunction

In [ ]:
#check classicality of conjunction
goal = read_expr(r'((exists x. (P(x) & Actual(x))) & (exists y. (Q(y) & Actual(y)))) <-> exists y exists z.(Actual(y) & Actual(z) & P(y) & Q(z) & Actual(Sum(y, z)))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

In [ ]:
#check classicality of conjunction
goal = read_expr(r'((exists x. (P(x) & Actual(x))) & (exists y. (Q(y) & Actual(y)))) <-> exists y exists z.(Actual(y) & Actual(z) & P(y) & Q(z) & Actual(Sum(y, z)))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

### Classicality of disjunction

In [ ]:
#check classicality of disjunction
goal = read_expr(r'(exists x. (Actual(x) & (P(x) | Q(x)))) <-> ((exists x. (Actual(x) & P(x))) & (exists y. (Actual(y) & Q(y))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

### No Gaps

In [32]:
#define negating a set
#x negates P iff there is a function h such that x = ⊔{h(xi) ∣ xi ∈ P} and for all events 
# x1 ∈ P, h(x1) excludes some part of x1.
# 
# x negates P iff every x1 in P has a part y that excludes some part x2 of x and
# every part x3 of x
def negate(x, P):
    result = ('(all x1. (P(x1) -> (exists x2. Partof(x2,x) & exists y. Partof(y,x1) & Excl(x2, y)))) 
              & (all x3. ((Partof(x3,x) & (-exists x4. (Partof(x4,x) & Partof(x4,x3)))) -> (exists x5. (P(x5) & Excl(x5, x3))))) & all x6 all x7. ((x6 != x7) -> (exists x8 exists x9.(Partof(x8,x) & Partof(x9,x) & x8!=x9 & Excl(x6, x8) & Excl(x7, x9))))')
    return result

#check no gaps
goal = read_expr('(-exists x. (actual(x) & P(x))) <-> (exists y. (Actual(y) &'+negate('y', 'P')+'))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

Prover9LimitExceededException: (MAX_SECONDS)

### No Gluts

In [ ]:
#check no gluts
goal = read_expr('-((exists x. Actual(x) & P(x)) & (exists y. Actual(y) &'+negate('y', 'P')+'))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())